In [1]:
import requests
import json
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
from itertools import product
import seaborn as sns
from scipy.stats import bernoulli
from scipy.stats import multivariate_normal
from random import random
import copy

In [2]:
df = pd.read_csv('../data/spotify_dataset.csv', usecols=range(4), lineterminator='\n', header=0)

In [3]:
df.columns = [x.replace('"', '').lstrip() for x in df.columns]

In [4]:
playlists = df['playlistname'].unique()
print('Number of Playlists:', len(playlists))
print('Number of Artists:', len(df['artistname'].unique()))

Number of Playlists: 161530
Number of Artists: 290002


#### Problem:
- Networkx takes too much time to load up a graph with all the data <br>
- In ~5 min we can retrieve 2000 artist genres


#### Potential Solution:
- Sample 1,000 playlists
- See if there is a sufficient amount of artists and playlist appearances

In [5]:
np.random.seed(0)
sample_size = 2000
sample_playlists = np.random.choice(playlists, sample_size, replace=False)
sampled_df = df[df['playlistname'].isin(sample_playlists)]

In [6]:
print('Number of songs in sample:', len(sampled_df))
print('Number of Artists:', len(sampled_df['artistname'].unique()))

Number of songs in sample: 126537
Number of Artists: 18139


In [7]:
appearances = sampled_df.groupby('artistname').agg({'trackname':'count', 'playlistname':lambda x: len(x.unique())})
appearances.sort_values(by=['trackname', 'playlistname'])

trackname  playlistname
artistname                                                   
!!!                                           1             1
!Tang                                         1             1
(LinSMHPAS)                                   1             1
(hed) pe/Kottonmouth Kings/Tech N9ne          1             1
009 Sound System                              1             1
...                                         ...           ...
Tony Bennett                                567             8
Marvin Gaye                                 599            35
The xx                                      631            41
Barenaked Ladies                            816            11
U2                                         4010            35

[18138 rows x 2 columns]

- There are some artists that appear on an insignificant amount of playlists, define this to be anything less than 10

In [74]:
artists = appearances[appearances['playlistname']>=10].index
print('# of artists on >= 10 playlists (sample):', len(artists))

# of artists on >= 10 playlists (sample): 623


In [46]:
df1 = sampled_df[sampled_df['artistname'].isin(artists)]
df1.head(10)

user_id         artistname  \
1893  944c80d26922ae634d6ce445b1fdff7f          Jake Bugg   
1897  944c80d26922ae634d6ce445b1fdff7f      Daniel Powter   
1898  944c80d26922ae634d6ce445b1fdff7f     Kelly Clarkson   
1903  944c80d26922ae634d6ce445b1fdff7f           Coldplay   
1904  944c80d26922ae634d6ce445b1fdff7f          Passenger   
1910  944c80d26922ae634d6ce445b1fdff7f    Lady Antebellum   
1913  944c80d26922ae634d6ce445b1fdff7f     Matt Nathanson   
1914  944c80d26922ae634d6ce445b1fdff7f  A Great Big World   
1915  944c80d26922ae634d6ce445b1fdff7f              Oasis   
1916  944c80d26922ae634d6ce445b1fdff7f        Mat Kearney   

                             trackname playlistname  
1893                 A Song About Love       JOOTDR  
1897                           Bad Day       JOOTDR  
1898                    Because of You       JOOTDR  
1903                           Fix You       JOOTDR  
1904  Golden Thread (feat. Matt Corby)       JOOTDR  
1910                      Need You Now       JOOTDR  
1913             Run - feat. Sugarland       JOOTDR  
1914                     Say Something       JOOTDR  
1915        Stop Crying Your Heart Out       JOOTDR  
1916                     Straight Away       JOOTDR

- Only take playlists that have more than 1 artist (otherwise there's no edge)

In [47]:
df1_grped = df1.groupby('playlistname').agg({'artistname':lambda x: len(x.unique())})
playlists = df1_grped[df1_grped['artistname'] > 1].index

In [48]:
df2 = df1[df1['playlistname'].isin(playlists)]
df2.head(10)

user_id         artistname  \
1893  944c80d26922ae634d6ce445b1fdff7f          Jake Bugg   
1897  944c80d26922ae634d6ce445b1fdff7f      Daniel Powter   
1898  944c80d26922ae634d6ce445b1fdff7f     Kelly Clarkson   
1903  944c80d26922ae634d6ce445b1fdff7f           Coldplay   
1904  944c80d26922ae634d6ce445b1fdff7f          Passenger   
1910  944c80d26922ae634d6ce445b1fdff7f    Lady Antebellum   
1913  944c80d26922ae634d6ce445b1fdff7f     Matt Nathanson   
1914  944c80d26922ae634d6ce445b1fdff7f  A Great Big World   
1915  944c80d26922ae634d6ce445b1fdff7f              Oasis   
1916  944c80d26922ae634d6ce445b1fdff7f        Mat Kearney   

                             trackname playlistname  
1893                 A Song About Love       JOOTDR  
1897                           Bad Day       JOOTDR  
1898                    Because of You       JOOTDR  
1903                           Fix You       JOOTDR  
1904  Golden Thread (feat. Matt Corby)       JOOTDR  
1910                      Need You Now       JOOTDR  
1913             Run - feat. Sugarland       JOOTDR  
1914                     Say Something       JOOTDR  
1915        Stop Crying Your Heart Out       JOOTDR  
1916                     Straight Away       JOOTDR

In [49]:
unq_play = sampled_df.groupby(['artistname', 'playlistname']).count().reset_index().groupby('artistname').count()[['playlistname']].reset_index()
# unq_play = unq_play.groupby('playlistname').count()
unq_play.head(10)


artistname  playlistname
0                                           !!!             1
1                                         !Tang             1
2                    #1 Katy Perry Tribute Band             1
3                                    (Hed) P.E.             1
4                                   (LinSMHPAS)             1
5          (hed) pe/Kottonmouth Kings/Tech N9ne             1
6                                           +44             1
7  ...And You Will Know Us By The Trail Of Dead             2
8                              009 Sound System             1
9                                           047             1

In [50]:
top_10 = unq_play.sort_values(by = 'playlistname', ascending = False)[:10]
f, ax = plt.subplots()
sns.set_color_codes("pastel")
sns.barplot(y="artistname", x="playlistname", data=top_10, orient = 'h', color = '#1f77b4')
ax.set(xlabel = 'frequency', title = 'Top 10 artist frequencies')

[Text(0.5, 0, 'frequency'), Text(0.5, 1.0, 'Top 10 artist frequencies')]

In [51]:
plt.hist(unq_play[(unq_play['playlistname'] > 1)]['playlistname'], bins=10)
plt.xlabel('Distinct Num of Playlists')
plt.ylabel('Freq of Artists')
plt.title('Total Playlist Appearances per Artist')
plt.savefig('playlistapp')

In [52]:
# same def as quarter 1
def read_edge(gph, n0, n1):
    if gph.has_edge(n0, n1):
        gph[n0][n1]['weight'] +=1
        return gph[n0][n1]['weight']
    else:
        gph.add_edge(n0, n1, weight=1)
        return 1

In [53]:
%%time
G = nx.Graph()
# if not unique, could weight the number of times the artist appears in that playlist
df2_grp = df2.groupby('playlistname').agg({'artistname': lambda x: (x).unique()})

for i in range(len(df2_grp)):
    
    for a in (df2_grp.iloc[i]):
        for a1, a2 in itertools.combinations(a, 2):
            read_edge(G, a1, a2)


CPU times: user 1.52 s, sys: 215 ms, total: 1.73 s
Wall time: 1.76 s


In [54]:
# threshold = 1
# long_edges = list(filter(lambda e: e[2] <= threshold, (e for e in G.edges.data('weight'))))
# G.remove_edges_from(long_edges)

In [55]:
%%time
nx.draw(G, node_size=50)


CPU times: user 26 s, sys: 879 ms, total: 26.9 s
Wall time: 27.4 s


In [56]:
# # [n for n in G.neighbors]
# neighbors = []
# for i in G.nodes:
#     count = 0
#     for k in G.neighbors(i):
#         count += 1
#     neighbors.append(count)
# neigh_df = pd.DataFrame(G.nodes)
# neigh_df['neighbors'] = neighbors
# neigh_df
# plt.hist(neigh_df['neighbors'])
# plt.xlabel('Number of Neighbors')
# plt.ylabel('Node Count')
# plt.title('Node Freq of Neighbor Count')
# plt.savefig('node_freq')

In [57]:
weight_dict = {}
node_weights = nx.get_edge_attributes(G, "weight")
for i in node_weights:
    if 'Justin Timberlake' in i:
        weight_dict[i] = node_weights[i]

In [58]:
sorted(weight_dict.items(), key=lambda x:x[1], reverse=True)[:10]

[(('JAY Z', 'Justin Timberlake'), 24),
 (('Beyoncé', 'Justin Timberlake'), 20),
 (('Rihanna', 'Justin Timberlake'), 20),
 (('Katy Perry', 'Justin Timberlake'), 19),
 (('Daft Punk', 'Justin Timberlake'), 18),
 (('Usher', 'Justin Timberlake'), 17),
 (('Kanye West', 'Justin Timberlake'), 17),
 (('Coldplay', 'Justin Timberlake'), 15),
 (('Maroon 5', 'Justin Timberlake'), 15),
 (('Justin Timberlake', 'Britney Spears'), 15)]

In [59]:
len(G.nodes)

1609

In [60]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def get_artist_genres(artist_names):
    client_id = '8ac5bd4f29854a4abb37648a5b7833d8'
    client_secret = 'c9f1281a0f774d97ae7d002139e53049'
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    artist_dict = {}
    
    for artist in artist_names:
        result = sp.search(artist, type='artist')['artists']['items']
        if len(result) > 0:
            artist_dict[artist] = result[0]['genres']
    return artist_dict


In [61]:
genres = get_artist_genres(list(G.nodes))

In [62]:
# dare's algorithm work don't worry about it
# def L_g_grad(u, F, A):
#     sum1 = 0
#     sum2 = 0
#     for edge in range(len(A[u])):
#         if A[u][edge] > 0:
#             print(-F[u] * F[edge].T, -F[u].dot(F[edge].T))
#             sum1 += np.log(1-np.exp(-F[u].dot(F[edge].T)))
#         else:
#             sum2 += F[u] * F[edge].T
#     return sum1 - sum2

# def train(A, Att, C, iterations = 100):
#     # initialize an F
#     N = A.shape[0]
#     F = np.random.rand(N,C)
#     X = Att
#     # weight matrix is (# of attributes, # of groups)
#     W = np.zeros((Att.shape[1], C))
#     F_new = F#np.zeros((N,C))
#     alpha = 0.01
#     ll = .01
#     ll_new = 0
#     W_new = W
#     for n in range(iterations):
#         dist = 0
#         F_old = F_new
#         for u in range(N):
#             L_g = (L_g_grad(u, F_new, A))
#             print(L_g, )
#             L_g = gradient(F, A, u) # derivative of L_g w/ respect to F_u (does this need to be F_new???)
#             Q = np.divide(1, 1 + np.exp(-W_new.dot(F_new[u].T)))
#             print(F_new[u])
#             L_x = alpha*((X[u]-Q).dot(W)) # derivative of L_x w/ respect to F_u
#             print(F_new[u], (np.zeros(C) < F_new[u] + alpha*(L_g+L_x)) * (F_new[u] + alpha*(L_g+L_x)))
#             F_new[u] = (np.zeros(C) < F_new[u] + alpha*(L_g+L_x)) * 1 * (F_new[u] + alpha*(L_g+L_x))
#             print(L_g,L_x)
#             W_new += alpha*(np.array(X[u]-Q)*(F_new[u])) - np.sign(W_new)


#         F_new += alpha*(X-Q).dot(W)
#         F = np.maximum(.01, F_new) # should this be 0?
#         ll_new = log_likelihood(F, A)
#         ll_new += np.sum(X*np.log(np.maximum(.01, Q)) + (1 - X)*np.log(np.maximum(.01, 1 - Q)))
#         change = (ll - ll_new) / ll
#         print(alpha*(np.dot((X-Q).T, F)) - np.sign(W_new))
#         if abs(change) < .001:
#             break
#         else:
#             ll = ll_new
#     print(n, W_new - np.sign(W_new))
#     delta = (-np.log(1- (1/N)))**.5
#     return F>delta, W_new

In [63]:
# implement algorithm
def standardize_matrix(A):
    max_edge = 0
    for i in nx.to_numpy_array(G):
        if (max(i)) > max_edge:
            max_edge = max(i)
    stand_A = []
    for row in nx.to_numpy_array(G):
        stand_row = []
        for edge in row:
            stand_row.append(edge/max_edge)
        stand_A.append(stand_row)
    return np.array(stand_A)


In [64]:
## algorithm work

# bigclam algorithm taken from https://github.com/RobRomijnders/bigclam
def sigm(x):
    return np.divide(np.exp(-1.*x),1.-np.exp(-1.*x))

def log_likelihood(F, A):
    """implements equation 2 of 
    https://cs.stanford.edu/people/jure/pubs/bigclam-wsdm13.pdf"""
    A_soft = F.dot(F.T)

    # Next two lines are multiplied with the adjacency matrix, A
    # A is a {0,1} matrix, so we zero out all elements not contributing to the sum
    FIRST_PART = A*np.log(1.-np.exp(-1.*A_soft))
    sum_edges = np.sum(FIRST_PART)
    SECOND_PART = (1-A)*A_soft
    sum_nedges = np.sum(SECOND_PART)

    log_likeli = sum_edges - sum_nedges
    return -log_likeli

def gradient(F, A, i):
    """Implements equation 3 of
    https://cs.stanford.edu/people/jure/pubs/bigclam-wsdm13.pdf
    
      * i indicates the row under consideration
    
    The many forloops in this function can be optimized, but for
    educational purposes we write them out clearly
    """
    N, C = F.shape

    neighbours = np.where(A[i])
    nneighbours = np.where(1-A[i])

    sum_neigh = np.zeros((C,))
    for nb in neighbours[0]:
        dotproduct = F[nb].dot(F[i])
        sum_neigh += F[nb]*sigm(dotproduct)

    sum_nneigh = np.zeros((C,))
    #Speed up this computation using eq.4
    for nnb in nneighbours[0]:
        sum_nneigh += F[nnb]

    grad = sum_neigh - sum_nneigh
    return grad

def train(A, Att, C, iterations = 100, alpha = .005, lambda_W = .001):
    # initialize an F
    N = A.shape[0]
    F = np.random.rand(N,C)
    F_new = np.zeros((N,C))
    X = Att
    W = np.random.rand(Att.shape[1], C)
    W_new = np.zeros((Att.shape[1], C))
    ll = .01
    ll_new = 0
    Q = np.zeros((F.shape[0], W.shape[0]))
    Sanity = np.zeros((F.shape[0], W.shape[0]))
    for n in range(iterations):
        for u in range(Q.shape[0]):
            for k in range(Q.shape[1]):
                running_sum = 0
                for c in range(F.shape[1]):
                    running_sum += W[k][c] * F[u][c]
                Q[u][k] = 1/(1+np.exp(-running_sum))
        
        for u in range(X.shape[0]):
            grad = gradient(F, A, u)
            for c in range(F.shape[1]):
                running_sum = 0
                for k in range(X.shape[1]):
                    running_sum += (X[u][k] - Q[u][k]) * W[k][c]
                F_new[u][c] = max(.001, F[u][c] + alpha*(running_sum + grad[c]))
        
        for k in range(X.shape[1]):
            for c in range(F.shape[1]):
                running_sum = 0
                for u in range(F.shape[0]):
                    running_sum += (X[u][k] - Q[u][k]) * F[u][c]
                W_new[k][c] = W[k][c] + alpha * running_sum - alpha * lambda_W * np.sign(W[k][c])

        F = copy.deepcopy(F_new)
        W = copy.deepcopy(W_new)
        ll_new = log_likelihood(F, A)
        ll_new += np.sum(np.maximum(.001, X*np.log(np.maximum(.001, Q)) + (1 - X)*np.log(np.maximum(.001, 1 - Q))))
        change = (ll - ll_new) / ll
        if abs(change) < .001:
            break
        else:
            ll = ll_new
    delta = (-np.log(1 - (1/N)))**.5
    
    return F, delta, W

In [65]:
# define attributes and group size
most_appearances = df2.groupby('artistname').agg({'playlistname':lambda x: len(np.unique(np.array(list(x))))}).max()
att = df2.groupby('artistname').agg({'playlistname':lambda x: len(np.unique(np.array(list(x)))) >= .75*most_appearances}) * 1
att = np.array([pd.Series([node for node in G.nodes]).apply(lambda x: dict(zip(att.index, att.playlistname))[x])]).T
c = 7 # group size



In [66]:
# try w/ binary adjacency
nodes = [node for node in G]
A = (nx.to_numpy_array(G) >= 1) * 1
iterations = 100
F_bin, delta, W = train(A, att, c, iterations)
print('sample size is: ' + str(sample_size), ', group size: ' + str(c), ', iterations: ' + str(iterations))
print(np.argmax(F_bin, 1))
print(W)

/Users/darehunt/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: RuntimeWarning: overflow encountered in exp


sample size is: 2000 , group size: 7 , iterations: 100
[5 5 5 ... 5 5 5]
[[-1741.72800306 -1741.10843955 -1741.97057234 -1741.16139651
  -1741.99526722 -1742.3118707  -1741.52046741]]


In [67]:
# # try stardardized adjacency
# nodes = [node for node in G]
# A = nx.to_numpy_array(G)
# A = standardize_matrix(A)
# iterations = 100
# F, delta, W = train(A, att, c)
# print('sample size is: ' + str(sample_size), ', group size: ' + str(c), ', iterations: ' + str(iterations))
# print(np.argmax(F, 1))
# print(W)

In [68]:
nodes_pred_bin = dict(zip(nodes, np.argmax(F_bin,1)))
# nodes_pred = dict(zip(nodes, np.argmax(F,1)))


In [69]:
# calc percent of top edges are in the same group
dict_nodes_edg_grp = {}
node_weights = nx.get_edge_attributes(G, "weight")
for i in nodes:
    dict_nodes_edg_grp[i] = []
    weight_dict = {}
    for j in node_weights:
        if i in j:
            weight_dict[j] = (node_weights[j], [nodes_pred_bin[j[0]], nodes_pred_bin[j[0]]])
    weight_dict = sorted(weight_dict.items(), key=lambda x:x[1], reverse=True)
    dict_nodes_edg_grp[i].append(weight_dict[:3])
    

In [70]:
# sorted(nodes_pred.items(), key=lambda x:x[1], reverse=True)

In [71]:
sorted(nodes_pred_bin.items(), key=lambda x:x[1], reverse=True)

[('I LOVE MAKONNEN', 6),
 ('Mando Diao', 6),
 ('Madeon', 6),
 ('Twin Shadow', 6),
 ('CSS', 6),
 ('Lit', 6),
 ('Lisa Gerrard', 6),
 ('RJD2', 6),
 ('Sugar Ray', 6),
 ('Gwen Stefani', 6),
 ('The Morning Benders', 6),
 ('Sak Noel', 6),
 ('Shinedown', 6),
 ('The Guess Who', 6),
 ('Alpha Blondy', 6),
 ('Michael Bolton', 6),
 ('Kenny Chesney', 6),
 ('Jungle', 6),
 ('Dvbbs', 6),
 ('Dimitri Vegas', 6),
 ('Sister Hazel', 6),
 ('Barenaked Ladies', 6),
 ('Azure Ray', 6),
 ('Sun Kil Moon', 6),
 ('Damien Jurado', 6),
 ('Cults', 6),
 ('George Clinton', 6),
 ('Slightly Stoopid', 6),
 ('Guster', 6),
 ('Washed Out', 6),
 ('Baths', 6),
 ('Deerhunter', 6),
 ('The Dodos', 6),
 ('Cut Copy', 6),
 ('Mount Kimbie', 6),
 ('Dave Matthews Band', 6),
 ('Beach House', 6),
 ('Mastodon', 6),
 ('Fabolous', 6),
 ('Panda Bear', 6),
 ('Lady Sovereign', 6),
 ('The Roots', 6),
 ('Animal Collective', 6),
 ('Clipse', 6),
 ('Girls', 6),
 ('Neko Case', 6),
 ('The Darkness', 6),
 ('The Walkmen', 6),
 ('Sugababes', 6),
 ('Woods'

In [72]:
weight_dict = {}
node_weights = nx.get_edge_attributes(G, "weight")
for i in node_weights:
    if 'Phoenix' in i:
        weight_dict[i] = (node_weights[i], [nodes_pred_bin[i[0]], nodes_pred_bin[i[1]]])
        

In [73]:
sorted(weight_dict.items(), key=lambda x:x[1], reverse=True)

[(('Phoenix', 'Daft Punk'), (12, [5, 1])),
 (('Phoenix', 'Foster The People'), (11, [5, 5])),
 (('Phoenix', 'Justin Timberlake'), (11, [5, 5])),
 (('Phoenix', 'Vampire Weekend'), (11, [5, 5])),
 (('Phoenix', 'MGMT'), (11, [5, 5])),
 (('Phoenix', 'Kings Of Leon'), (10, [5, 5])),
 (('Phoenix', 'Two Door Cinema Club'), (10, [5, 5])),
 (('Phoenix', 'JAY Z'), (10, [5, 5])),
 (('Phoenix', 'The Naked And Famous'), (10, [5, 5])),
 (('Phoenix', 'Bloc Party'), (9, [5, 5])),
 (('Phoenix', 'The Black Keys'), (9, [5, 5])),
 (('Phoenix', 'Katy Perry'), (9, [5, 5])),
 (('Phoenix', 'Arcade Fire'), (9, [5, 5])),
 (('Passion Pit', 'Phoenix'), (8, [5, 5])),
 (('Phoenix', 'The xx'), (8, [5, 5])),
 (('Phoenix', 'Modest Mouse'), (8, [5, 5])),
 (('Phoenix', 'Kanye West'), (8, [5, 5])),
 (('Phoenix', 'M83'), (8, [5, 5])),
 (('Phoenix', 'The Strokes'), (8, [5, 5])),
 (('Phoenix', 'Disclosure'), (8, [5, 5])),
 (('Phoenix', 'Coldplay'), (8, [5, 1])),
 (('Phoenix', 'Beach House'), (7, [5, 6])),
 (('Phoenix', 'The

In [ ]:
[x for x in nodes_pred if nodes_pred[x]==2]

In [ ]:
# Node attributes

In [ ]:
df2
# avg songs per playlist
x = df2.groupby('artistname').agg({'playlistname':lambda x: len(np.unique(np.array(list(x)))), 'trackname':'count'})
x['trackname'] / x['playlistname']


In [ ]:
# total songs
x = df2.groupby('artistname').agg({'trackname':'count'})
x


In [ ]:
# song variety - closer to 1: unique song on every playlist, closer to 0: many songs in common over playlists
x = df2.groupby('artistname').agg({'trackname':lambda x: len(np.unique(np.array(list(x)))), 'playlistname':'count'})
(x['trackname'] / x['playlistname']).sort_values()


In [ ]:
# playlist appearances
x = df2.groupby('artistname').agg({'playlistname':lambda x: len(np.unique(np.array(list(x))))})
x.sort_values(by='playlistname')
x['playlistname'].groupby(x['playlistname']).count()


In [ ]:
x.mean()


In [37]:
len(nodes)

623

In [44]:
sorted(genres.items(), key=lambda x:x, reverse=True)

[('will.i.am', ['dance pop', 'pop', 'pop rap', 'post-teen pop']),
 ('deadmau5',
  ['canadian electronic',
   'edm',
   'electro house',
   'pop dance',
   'progressive house']),
 ('blink-182', ['pop punk', 'punk', 'rock', 'socal pop punk']),
 ('alt-J', ['indie rock', 'modern rock']),
 ('a-ha',
  ['new romantic',
   'new wave',
   'new wave pop',
   'permanent wave',
   'pop rock',
   'soft rock',
   'synthpop']),
 ('Zero 7', ['downtempo', 'electronica', 'nu jazz', 'trip hop']),
 ('Zedd',
  ['complextro',
   'dance pop',
   'edm',
   'electro house',
   'electropop',
   'german techno',
   'pop',
   'pop dance',
   'pop rap',
   'post-teen pop',
   'tropical house']),
 ('ZZ Top',
  ['album rock',
   'blues rock',
   'classic rock',
   'country rock',
   'hard rock',
   'metal',
   'rock']),
 ('Young the Giant',
  ['modern alternative rock', 'modern rock', 'rock', 'stomp and holler']),
 ('Yeah Yeah Yeahs',
  ['alternative dance',
   'alternative rock',
   'dance-punk',
   'garage rock',
